In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from numpy.lib.shape_base import column_stack
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.core.tools import numeric
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History

In [ ]:
df_train = pd.read_csv('../input/rossmann-store-sales/train.csv', low_memory=False, parse_dates=['Date'])
df_store = pd.read_csv('../input/rossmann-store-sales/store.csv', header=0)
df_train.head()
df_store.head()

In [ ]:
df_new = df_train.merge(df_store, on=['Store'], how='inner')
df_new.head()

In [ ]:
df_new.shape

In [ ]:
df_new['Date'] = pd.to_datetime(df_new['Date'], infer_datetime_format=True)
df_new['Date']

In [ ]:
df_new['Week'] = df_new['Date'].dt.isocalendar().week
df_new['Week']

In [ ]:
df_new['Month'] = df_new['Date'].dt.month
df_new['Month']

In [ ]:
df_new['Year'] = df_new['Date'].dt.year
df_new['Year']

In [ ]:
df_new['Quarter'] = df_new['Date'].dt.quarter
df_new['Quarter']

In [ ]:
df_new['Day'] = df_new['Date'].dt.day
df_new['Day']

In [ ]:
df_new['Season'] = np.where(df_new['Month'].isin([3, 4, 5]), 'Spring',
                    np.where(df_new['Month'].isin([6, 7, 8]), 'Summer',
                    np.where(df_new['Month'].isin([9, 10, 11]), 'Fall',
                    np.where(df_new['Month'].isin([12, 1, 2]), 'Winter', 'None'))))

In [ ]:
df_new['CompetitionDistance'] = df_new['CompetitionDistance'].fillna(df_new['CompetitionDistance'].mode()[0])
df_new['CompetitionDistance']

In [ ]:
target = ['Sales']
numeric_columns = ['Customers', 'Open', 'Promo', 'Promo2', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance']
categorical_columns = ['DayOfWeek', 'Quarter', 'Month', 'Year', 'StoreType', 'Assortment', 'Season']

In [ ]:
def create_ohe(df, col):
    le = LabelEncoder()
    a = le.fit_transform(df_new[col]).reshape(-1, 1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col + ' ' + str(i) for i in le.classes_]
    return pd.DataFrame(ohe.fit_transform(a), columns=column_names)

In [ ]:
temp = df_new[numeric_columns]
for column in categorical_columns:
    temp_df = create_ohe(df_new, column)
    temp = pd.concat([temp, temp_df], axis=1)

In [ ]:
print(temp['StateHoliday'].unique())
temp['StateHoliday'] = np.where(temp['StateHoliday']=='0', 0, 1)
print(temp.dtypes.unique())

x_train, x_test, y_train, y_test = train_test_split(temp, df_new[target], test_size=0.2, random_state=2018)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=2018)

print('Shape of x_train:{0}'.format(x_train.shape))
print('Shape of x_val:{0}'.format(x_val.shape))
print('Shape of x_test:{0}'.format(x_test.shape))
print('Shape of y_train:{0}'.format(y_train.shape))
print('Shape of y_val:{0}'.format(y_val.shape))
print('Shape of y_test:{0}'.format(y_test.shape))

In [ ]:
mean_scales = y_train.mean()
print('Average Sales:{0}'.format(mean_scales))
print('MAE for Test Data:{0}'.format(abs(y_test - mean_scales).mean()[0]))

In [ ]:
history = History()
model = Sequential()
model.add(Dense(350, input_dim=44, activation='relu'))
model.add(Dense(350, activation='relu'))
model.add(Dense(350, activation='relu'))
model.add(Dense(350, activation='relu'))
model.add(Dense(350, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
model.fit(x_train.values, y_train.values, validation_data=(x_val, y_val), epochs=11, batch_size=64, callbacks=[history])

In [ ]:
result = model.evaluate(x_test.values, y_test.values)
for i in range(len(model.metrics_names)):
    print('Metrics ', model.metrics_names[i], ':', str(round(result[i], 2)))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model's Training & Validation loss across epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
y_test['Prediction'] = model.predict(x_test)
y_test.columns = ['Actual Sales', 'Sales']
print(y_test.head(10))

print('MSE:{0}'.format(mean_squared_error(y_test['Actual Sales'].values, y_test['Sales'].values)))
print('MAE:{0}'.format(mean_absolute_error(y_test['Actual Sales'].values, y_test['Sales'].values)))

In [ ]:
y_test.shape
y_test

In [ ]:
y_test.index.set_names(['Id'], inplace=True)
y_test

In [ ]:
y_test.drop(['Actual Sales', 'Prediction'], axis=1, inplace=True)
y_test

In [ ]:
y_test.rename(columns={'Predicted Sales': 'Sales'})

In [ ]:
y_test.to_csv('submission.csv')

### end